In [18]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests
import json
import ipyvuetify as viewtify

my_headers = {'accept' : 'application/json'}

# Initialize the quant_dd content by calling the quantity kind request, parsing the resulting json, and acquiring
# the results and bindings elements of the response.

quant_response = requests.get("https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?qk ?qkl WHERE { ?qk a qudt:QuantityKind . ?qk rdfs:label ?qkl . }", headers=my_headers)
qdata          = quant_response.json()
qresults       = qdata['results']
qbinds         = qresults['bindings']

# get_qvalues: Parses the 'qk' value array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qlabels - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_qvalues(obj):
    qvalues = []
    i       = 0

    for item in obj:
        elem = item['qk']
        qvalues.append(elem['value'])
        i+=1

    return qvalues

quant_values = get_qvalues(qbinds)

# get_qlabels: Parses the 'qkl' label array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qlabels - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_qlabels(obj):
    qlabels = []
    i       = 0

    for item in obj:
        elem = item['qkl']
        qlabels.append(elem['value'])
        i+=1

    return qlabels

quant_labels = get_qlabels(qbinds)

# Initialize the app title, help Text widget and start_value Text widget

file         = open("../images/qudt_logo-300x110.png", "rb")
image        = file.read()
label_style  = {'description_width' : 'initial'}
title_text   = viewtify.Html(tag = "div", children = ['Unit Conversion Tool'], style_ = " color: 'colors.indigo.darken4'; font-family: helvetica; font-size: 20px")
titleBox     = widgets.HBox([widgets.Image(value = image, format = 'png', width = 75, height = 27), title_text])
help_label   = widgets.HTML(value = '<a href = "../help/help.html"><b>Help</b></a>')
start_value  = widgets.Text(value = '1', description = 'Value to convert', style = label_style, disabled = False)

# Construct the quant_dd Dropdown widget

quant_dd = widgets.Dropdown(
    options     = quant_labels,
    description = 'Desired Quantity:',
    style       = label_style,
    disabled    = False,
)

# start_units_get_request: Constructs and issues the start unit SPARQL query, converts to json, and parses
#                          the ressulting results and bindings values. The bindings are returned.
#
# ARGUMENTS: selected_quant - the currently selected quantity
# RETURN:    stubinds      - the SPARQL query bindings array
# CALLED BY: update_options_from_selected_quantity

def start_units_get_request(selected_quant):
    stunit_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?stunit ?stunit_label WHERE { <"
    stunit_query_quant = quant_values[quant_dd.index]
    stunit_query_end   = "> qudt:applicableUnit ?stunit . ?stunit rdfs:label ?stunit_label . }"
    stunit_query       = stunit_query_start + selected_quant + stunit_query_end

    stunit_response = requests.get(stunit_query, headers = my_headers)
    studata         = stunit_response.json()
    sturesults      = studata['results']
    stubinds        = sturesults['bindings']

    return stubinds

# get_stuvalues: Parses the 'stunit' value array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    stuvalues - the starting unit SPARQL query values array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_stuvalues(obj):
    stuvalues = []
    i         = 0

    for item in obj:
        elem = item['stunit']
        stuvalues.append(elem['value'])
        i += 1

    return stuvalues

# get_stulabels: Parses the 'stunit_label' value array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    stulabels - the start unit SPARQL query labels array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_stulabels(obj):
    stulabels = []
    i         = 0

    for item in obj:
        elem = item['stunit_label']
        stulabels.append(elem['value'])
        i += 1

    return stulabels

# Initialize the start_unit_dd content (options) by calling the request and constructing the values and labels arrays

stubinds      = start_units_get_request(quant_values[quant_dd.index])
stunit_values = get_stuvalues(stubinds)
stunit_labels = get_stulabels(stubinds)

# Construct the start_unit_dd Dropdown widget

start_unit_dd = widgets.Dropdown(
    options     = stunit_labels,
    description = 'Starting Unit:',
    style       = label_style,
    disabled    = False,
)

# target_units_get_request: Constructs and issues the target unit SPARQL query, converts to json, and parses
#                           the ressulting results and bindings values. The bindings are returned.
#
# ARGUMENTS: selected_quant - the currently selected quantity
#            stunits        - the start_unit_dd values array
# RETURN:    tgtubinds      - the SPARQL query bindings array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def target_units_get_request(selected_quant, stunits):
    tgtunit_query_start  = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/> PREFIX unit: <http://qudt.org/vocab/unit/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?tgtunit ?tgtunit_label WHERE { <"
    unit_query_quant     = quant_values[quant_dd.index]
    tgtunit_query_middle = "> qudt:applicableUnit ?tgtunit . ?tgtunit rdfs:label ?tgtunit_label . FILTER (?tgtunit != <"
    stunit_value         = stunits[start_unit_dd.index]
    tgtunit_query_end    = ">) . }"
    tgtunit_query        = tgtunit_query_start + selected_quant + tgtunit_query_middle + stunit_value + tgtunit_query_end

    tgtunit_response = requests.get(tgtunit_query, headers = my_headers)
    tgtudata         = tgtunit_response.json()
    tgturesults      = tgtudata['results']
    tgtubinds        = tgturesults['bindings']

    return tgtubinds

# get_tgtuvalues: Parses the 'tgtunit' value array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    stuvalues - the starting unit SPARQL query values array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_tgtuvalues(obj):
    tgtuvalues = []
    i          = 0

    for item in obj:
        elem = item['tgtunit']
        tgtuvalues.append(elem['value'])
        i += 1

    return tgtuvalues

# get_tgtulabels: Parses the 'tgtunit_label' value array from the SPARQL query results.
#
# ARGUMENTS: obj        - a bindings array
# RETURN:    tgtulabels - the target unit SPARQL query labels array
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit

def get_tgtulabels(obj):
    tgtulabels = []
    i          = 0

    for item in obj:
        elem = item['tgtunit_label']
        tgtulabels.append(elem['value'])
        i += 1

    return tgtulabels

# Initialize the quant_dd content (options) by calling the request and constructing the values and labels arrays

tgtubinds      = target_units_get_request(quant_values[quant_dd.index], stunit_values)
tgtunit_values = get_tgtuvalues(tgtubinds)
tgtunit_labels = get_tgtulabels(tgtubinds)

# Construct the quant_dd Dropdown widget

target_unit_dd = widgets.Dropdown(
    options     = tgtunit_labels,
    description = 'Target Unit:',
    style       = label_style,
    disabled    = False,
)

# starting_unit_cm_get_request: Constructs and calls the starting unit conversion multiplier SPARQL query and parses
#                               the results (and returns them). If there are no 'bindings' (empty response), then
#                               set the conversion multiplier to 1.0.
#
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit
#            update_calculations_from_selected_start_value
# CALLS:     get request method
#            json parse

def starting_unit_cm_get_request(selected_stu):
    stunitcm_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX unit: <http://qudt.org/vocab/unit/> SELECT DISTINCT ?stunitcm WHERE { <"
    stunitcm_query_end   = "> qudt:conversionMultiplier ?stunitcm . }"
    stunitcm_query       = stunitcm_query_start + selected_stu + stunitcm_query_end
    stunitcm_response    = requests.get(stunitcm_query, headers=my_headers)
    stucmdata            = stunitcm_response.json()
    stucmresults         = stucmdata['results']
    stucmbinds           = stucmresults['bindings']
    if len(stucmbinds) > 0:
        stucmelem         = stucmbinds[0]
        stucm             = stucmelem['stunitcm']
        stucmvalue        = stucm['value']
        start_unit_cm     = stucmvalue
    else:
        start_unit_cm = 1.0

    return start_unit_cm

# target_unit_cm_get_request: Constructs and calls the target unit conversion multiplier SPARQL query and parses
#                             the results (and returns them). If there are no 'bindings' (empty response), then
#                             set the conversion multiplier to 1.0.
#
# CALLED BY: update_options_from_selected_quantity
#            update_options_from_selected_start_unit
#            update_calculations_from_selected_start_value
# CALLS:     get request method
#            json parse

def target_unit_cm_get_request(selected_tgtu):
    tgtunitcm_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX unit: <http://qudt.org/vocab/unit/> SELECT DISTINCT ?tgtunitcm WHERE { <"
    tgtunitcm_query_end   = "> qudt:conversionMultiplier ?tgtunitcm . }"
    tgtunitcm_query       = tgtunitcm_query_start + selected_tgtu + tgtunitcm_query_end
    tgtunitcm_response    = requests.get(tgtunitcm_query, headers=my_headers)
    tgtucmdata            = tgtunitcm_response.json()
    tgtucmresults         = tgtucmdata['results']
    tgtucmbinds           = tgtucmresults['bindings']
    if len(tgtucmbinds) > 0:
        tgtucmelem         = tgtucmbinds[0]
        tgtucm             = tgtucmelem['tgtunitcm']
        tgtucmvalue        = tgtucm['value']
        target_unit_cm     = tgtucmvalue
    else:
        target_unit_cm = 1.0

    return target_unit_cm

# Initializer start_unit_cm, target_unit_cm, converted_value, and target_value Text widget

start_unit_cm   = 1
target_unit_cm  = 1
converted_value = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
target_value    = widgets.Text(value = str(converted_value), description = 'Converted value', style = label_style, disabled=True)

# update_options_from_selected_quantity: Performs model updates downstream of a changed selection in the quant_dd
#                                        dropdown. That is, it updates the start_unit_dd and target_unit_dd 
#                                        model (options) contents, the conversion multipliers, and the converted 
#                                        value.
#
# CALLED BY: on_change_q
# CALLS:     start_units_get_request, get_stuvalues, get_stulabels
#            target_units_get_request, get_tgtuvalues, get_tgtulabels
#            starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_options_from_selected_quantity(*args): # *args represent zero (case here) or more arguments.
    selected_quant         = quant_values[quant_dd.index] # if quant_dd.index else 0]
    stubinds               = start_units_get_request(selected_quant)
    stunit_values          = get_stuvalues(stubinds)
    stunit_labels          = get_stulabels(stubinds)
    start_unit_dd.options  = stunit_labels
    tgtubinds              = target_units_get_request(selected_quant, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    selected_stu           = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm          = starting_unit_cm_get_request(selected_stu)
    selected_tgtu          = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm         = target_unit_cm_get_request(selected_tgtu)
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

# on_change variations: There are 4 on_change variants:
#
#     1. Changes to the quantity kind must be reflected in all downstream widget models/views
#     2. Changes to the starting unit must be reflected in the target unit and converted value
#     3. Changes to the target unit must be reflected in the converted value
#     4. Changes to the starting value must be reflected in the converted value
#
# on_change_q: If the selected quantity kind changes, then everything downstream must change.
#
# CALLED BY: Changes to quant_dd index
# CALLS: update_options_from_selected_quantity

def on_change_q(change):
    selected_quant  = quant_values[quant_dd.index if quant_dd.index else 0]

    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_quantity()

quant_dd.observe(on_change_q)

# update_options_from_selected_start_unit: Performs model updates downstream of a changed selection in the start_unit_dd
#                                          dropdown. That is, it updates the target_unit_dd contents, the conversion
#                                          multipliers, and the converted value.
#
# CALLED BY: on_change_q
# CALLS:     target_units_get_request, get_tgtuvalues, get_tgtulabels
#            starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_options_from_selected_start_unit(*args): # *args represent zero (case here) or more arguments.
    selected_quant         = quant_values[quant_dd.index] # if quant_dd.index else 0]
    stubinds               = start_units_get_request(selected_quant)
    stunit_values          = get_stuvalues(stubinds)
    tgtubinds              = target_units_get_request(selected_quant, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    selected_stu           = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm          = starting_unit_cm_get_request(selected_stu)
    selected_tgtu          = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm         = target_unit_cm_get_request(selected_tgtu)
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

# on_change_stu: Watches changes to start_unit_dd and calls update_options_from_selected_start_unit when the selected
#                index changes. It thus observes start_unit_dd.
#
# CALLED BY: Changes to start_unit_dd index values
# CALLS      update_options_from_selected_start_unit

def on_change_stu(change):
    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_start_unit()

start_unit_dd.observe(on_change_stu)

# update_options_from_selected_target_unit: Performs model updates downstream of a changed selection in the target_unit_dd
#                                           dropdown. That is, it updates the target_unit_dd contents, the conversion
#                                           multipliers, and the converted value.
#
# CALLED BY: on_change_q
# CALLS:     target_units_get_request, get_tgtuvalues, get_tgtulabels
#            starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_options_from_selected_target_unit(*args): # *args represent zero (case here) or more arguments.
    selected_quant         = quant_values[quant_dd.index] # if quant_dd.index else 0]
    stubinds               = start_units_get_request(selected_quant)
    stunit_values          = get_stuvalues(stubinds)
    tgtubinds              = target_units_get_request(selected_quant, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    selected_stu           = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm          = starting_unit_cm_get_request(selected_stu)
    selected_tgtu          = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm         = target_unit_cm_get_request(selected_tgtu)
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

# on_change_tgtu: Watches changes to target_unit_dd and calls update_options_from_selected_start_unit when the selected
#                 index changes. It thus observes target_unit_dd.
#
# CALLED BY: Changes to target_unit_dd index values
# CALLS      update_options_from_selected_target_unit

def on_change_tgtu(change):
    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_target_unit()

target_unit_dd.observe(on_change_tgtu)

# update_calculations_from_selected_start_value: Performs model updates downstream of a changed value in the start_unit_dd
#                                                dropdown. That is, it updates the conversion multipliers, and the converted value.
#
# CALLED BY: on_change_suv
# CALLS:     starting_unit_cm_get_request
#            target_unit_cm_get_request

def update_calculations_from_selected_start_value(*args): # *args represent zero (case here) or more arguments.
    selected_quant     = quant_values[quant_dd.index] # if quant_dd.index else 0]
    stubinds           = start_units_get_request(selected_quant)
    stunit_values      = get_stuvalues(stubinds)
    selected_stu       = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    start_unit_cm      = starting_unit_cm_get_request(selected_stu) # if starting_unit_cm_get_request() != 'None' else 1
    tgtubinds          = target_units_get_request(selected_quant, stunit_values)
    tgtunit_values     = get_tgtuvalues(tgtubinds)
    selected_tgtu      = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    target_unit_cm     = target_unit_cm_get_request(selected_tgtu)   # if target_unit_cm_get_request() != 'None' else 1
    converted_value    = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value = str(converted_value)

# on_changes_suv: Watches changes to start_value and updates conversion multipliers and converted values when it
#                 changes.
#
# CALLS: update_calculations_from_selected_start_value

def on_change_suv(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_calculations_from_selected_start_value()

start_value.observe(on_change_suv)

dyn_grid_items  = [quant_dd, help_label, start_unit_dd, target_unit_dd, start_value, target_value]
dyn_box         = widgets.GridBox(dyn_grid_items, layout=widgets.Layout(grid_template_columns="repeat(2,325px)"))
app_box         = widgets.VBox([titleBox, dyn_box])
display(app_box)
